<h1 style="text-align: center;">Coursera_Capstone</h1>

### This is the capstone project for the final course in Applied Data Science specialization

First, import the required modules, and print a welcome statement.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
